**Setup the application**

In [1]:
import glob, sys

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, O, M, fetch_tables, WikidataSemanticModelHelper
sys.path.append(str(ROOT_DIR / "sm_annotator"))

from sm_annotator.prelude import Annotator, BatchAnnotator, widgets, GRAMSAnnotatorAssistant, SliderApp, BasicApp, Session

# %matplotlib widget
# import matplotlib.pyplot as plt

In [2]:
cwd = ROOT_DIR / "examples/misc"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

**Fetch tables from a webpage and select table that we want to annotate**

In [10]:
# tables = fetch_tables("https://en.wikipedia.org/wiki/List_of_largest_selling_pharmaceutical_products")
tables = fetch_tables("https://en.wikipedia.org/wiki/2018_Colombian_presidential_election")

In [13]:
def display_df(df):
    display(df)

slider = SliderApp(BasicApp(display_df), 'set_data')
slider.render()

Output()

HTML(value='<div id="bb6a03ee-9fe4-4a6c-8b5c-d818fb2e016f"></div>')

Output()

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='b395786b-0ba8-4ceb-9b4a-8af49a01637a')

<IPython.core.display.Javascript object>

In [14]:
slider.set_data([
    {"description": "<br>" + " > ".join([r.header for r in table.context]), "args": [table.as_df()]}
    for table in tables
])

**Annotate the selected table**

In [15]:
selected_table = 11

table = tables[selected_table]
table = table.as_relational_linked_table(table_id=f"{table.page_url}/{selected_table}")
annotation = grams.annotate(table)

IndexError: list index out of range

In [7]:
(cwd / f"ground-truth/{table.get_friendly_fs_id()}").mkdir(exist_ok=True)
M.serialize_json(table.to_json(), cwd / f"tables/{table.get_friendly_fs_id()}.json")
M.serialize_json(Session(table.id, False, "", table, [WikidataSemanticModelHelper.minify_sm(annotation.sm)]).to_json(), 
                 cwd / f"ground-truth/{table.get_friendly_fs_id()}/version.01.json") 

fname = table.get_friendly_fs_id()
description = f"<b>context</b>" + " > ".join([
    f"<b>[h{h.level}]</b> {h.header.strip()}" 
    for h in tables[selected_table].context
])
annotate_data = [(fname, description, table)]
assistant_data = [{"table": table, "sg": annotation.sg, "dg": annotation.dg}]

In [8]:
qnodes = grams.qnodes.cache_dict()
wdprops = grams.wdprops
wdclasses = grams.wdclasses.cache_dict()
es_host = 'http://mira.isi.edu:9200'

assistant = GRAMSAnnotatorAssistant(assistant_data, qnodes, wdprops)
annotator = Annotator(qnodes, wdclasses, wdprops, cwd / "ground-truth", es_host, username='', password='', dev=False, assistant=assistant)
annotator.render(True)

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='271081d6-8437-4615-8342-bda19355fbd8')

HTML(value='<div id="b68105e2-d631-4674-8515-901bc1c51e97"></div>')

<IPython.core.display.Javascript object>

In [9]:
annotator.annotate(annotate_data[0][0], annotate_data[0][2])